In [1]:
import numpy as np # linear algebra
import pandas as pd 


In [10]:
train_df = pd.read_csv("data/data45260/Traindata.txt", sep='\t', names=['ClassNo','Sentence' , 'ClassName'])
test_df = pd.read_csv("data/data45260/Testdata.txt", sep='\t',names=['ClassNo','Sentence' , 'ClassName'])


In [3]:
print(train_df.head())
print(train_df.info())

    ClassNo                                           Sentence ClassName
0  20150131  Tony Abbott's gaffe in knighting the Queen's h...   Opinion
1  20150131  Has so much trouble ever come from something s...   Opinion
2  20150131  Blackabbott's clever plan to fix the budget an...   Opinion
3  20150131  Burning Credlin is not going to help solve the...   Opinion
4  20150130  Food As more local produce heads offshore and ...   Opinion
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38940 entries, 0 to 38939
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ClassNo    38940 non-null  int64 
 1   Sentence   38940 non-null  object
 2   ClassName  38940 non-null  object
dtypes: int64(1), object(2)
memory usage: 912.8+ KB
None


In [5]:
labels = train_df["ClassName"].unique()
labels

array(['Opinion', 'News', 'Companies and Markets'], dtype=object)

In [6]:
train_df["ClassName"].value_counts()

ClassName
News                     14073
Companies and Markets    12563
Opinion                  12304
Name: count, dtype: int64

In [7]:
test_df["ClassName"].value_counts()

ClassName
Companies and Markets    2105
News                     2093
Opinion                  1709
Name: count, dtype: int64

In [14]:
import os
import torch
import re
import copy
import torch.nn as nn
from torch.nn import Embedding,Conv1d
from pathlib import Path

import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader, random_split
from torch import nn,optim
from tqdm.notebook import tqdm

from torch.optim import Adam

In [15]:
embedding_dim = 300  # 设定需要的embedding长度
# word_dict是一个单词于id的映射字典，
# <pading>的意思是补0，<unk>代表不认识的单词，其实就是glove词向量中没有的单词都会被认为是<unk>
word_dict = {'<pading>':0,"<unk>": 1}


# 加载对应长度的glove预训练词向量，维度越大的词向量加载越慢，300维的词向量文件有1G

glove_df = pd.read_csv("data/glove.6B.300d.txt", sep=" ", quoting=3, header=None, index_col=0)
# 生成对应的字典形式，key为单词，value为词向量
glove_dict = {key: val.values for key, val in glove_df.T.items()}

In [16]:
def word_tokenize(text: str):
    """ 
    这是一个切分单词的函数，这个函数除了简单了分词之外，
    还会将word_dict补充完整，生成完整的词表映射
    """
    word_index = []
    pat = re.compile(r"[\w]+|[.,!?;|]") 
    tokens = pat.findall(text.lower())  
    for token in tokens:
        if token not in word_dict:
            word_dict[token] = len(word_dict) if token in glove_dict else word_dict["<unk>"]
        word_index.append(word_dict[token])
    return word_index
# 训练集和测试集分词
train_text = train_df["Sentence"].apply(lambda s: word_tokenize(str(s)))
test_text = test_df["Sentence"].apply(lambda s: word_tokenize(str(s)))
MAX_LENGTH = 1000  # 最大句子长度
train_text = train_text.apply(lambda x:(x+[0]*MAX_LENGTH)[:MAX_LENGTH])
test_text = test_text.apply(lambda x:(x+[0]*MAX_LENGTH)[:MAX_LENGTH])
s2i = {'News':0,'Opinion':1,'Companies and Markets':2}
# s2i = {'Accounting':1,'Chanticleer':2,'Companies and Markets':3,'Computers':4,'Domain Prestige':5,'Education':6,'Features':7,'Financial Services':8,'Life & Leisure':9,'Market Wrap':10,
#       'Marketing & Media':11,'News':12,'Opinion':13,'Perspective':14,'Property':15,'Review':16,'Saleroom':17,'Smart Investor':18,
#       'Stock Tables':19,'Supplement':20,'Weekend Fin':21,'World':0}
train_df['ClassName']=train_df['ClassName'].replace(s2i).astype(int)
test_df['ClassName']=test_df['ClassName'].replace(s2i).astype(int)

In [17]:
# 预训练的词向量矩阵，这个后面会直接输入embedding层
glove_embeddings = np.zeros((len(word_dict), embedding_dim))
for k, v in word_dict.items():
    if v==0:
        glove_embeddings[v] = np.zeros(embedding_dim)
    glove_embeddings[v] = glove_dict[k] if k in glove_dict else glove_dict["<unk>"]

In [72]:
class CommentDataset(Dataset):
    def __init__(self,texts,labels):
        self.texts=texts
        self.labels=labels
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self,item):
        """
        item 为数据索引，迭代取第item条数据
        """
        text=torch.tensor(self.texts[item],dtype=torch.long)
        label=torch.tensor(self.labels[item],dtype=torch.long)
        return {'text_id':text,'label':label}
def create_data_loader(X,y,batch_size):
    ds=CommentDataset(
        texts = X.values,
        labels=y.values
    )
    return DataLoader(
        ds,
        batch_size=batch_size
    )
BATCH_SIZE = 32
train_data_loader = create_data_loader(train_text,train_df['ClassName'], BATCH_SIZE)
val_data_loader = create_data_loader(test_text,test_df['ClassName'], BATCH_SIZE)

In [96]:
class BiLSTM(nn.Module):
    def __init__(self, weight, embedding_dim=300, hidden_size=128, num_layers=2, dropout=0.5):
        super(BiLSTM, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(weight, freeze=False)
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dropout = dropout
        self.rnn = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_size,
                           num_layers=num_layers, batch_first=True, bidirectional=True)
        self.dropout = nn.Dropout(0.5)
        self.out = nn.Linear(hidden_size, 3)
    def forward(self, X):
        X = self.embedding(X)
        output, _ = self.rnn(X)
        output = torch.cat((output[:, -1, :self.hidden_size], output[:, 0, self.hidden_size:]), dim=1)
        output = self.out(output)
        return output


In [44]:
from collections import defaultdict

history = defaultdict(list)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # 设置CUDA
N_EPOCHS = 30     # 设置模型训练次数
learning_rate = 3e-4  # 初始学习率
model_name = 'BiLSTM'
model= BiLSTM(torch.from_numpy(glove_embeddings).float())
model = model.to(device)
# 优化器
optimizer = Adam(model.parameters(),learning_rate)

# 损失
criterion = nn.CrossEntropyLoss().to(device)
best_acc = 0
for epoch in tqdm(range(N_EPOCHS)):
    #启用模型的训练模式
    model.train()
    # 定义损失
    epoch_loss = 0
    epoch_acc = 0
    val_number = 0
    for batch in train_data_loader:
        text_id = batch['text_id'].to(device)
        label = batch['label'].to(device)
        predictions = model(text_id)
        loss = criterion(predictions, label)
        epoch_loss += loss.item()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        preds = predictions.max(1)[1]
        epoch_acc += ((preds==label).sum().item())
        val_number += label.size(0)
    history[f'{model_name}_train_loss'].append(epoch_loss / len(train_data_loader))
    history[f'{model_name}_train_accuracy'].append(epoch_acc / val_number)
    print(f'第{epoch+1}轮，训练Loss：',epoch_loss / len(train_data_loader),'，训练准确率：',epoch_acc / val_number)
    

In [57]:
save_path = 'ju32BiLStmepoch30learnrate3e-4'
torch.save(model.state_dict(), save_path)


In [97]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # 设置CUDA
model_name = 'BiLSTM'
model= BiLSTM(torch.from_numpy(glove_embeddings).float())
model = model.to(device)
# 优化器
optimizer = Adam(model.parameters(),learning_rate)

# 损失
criterion = nn.CrossEntropyLoss().to(device)
best_acc = 0

In [98]:
load_path='pyBiLStmepoch30learnrate3e-4'
model.load_state_dict(torch.load(load_path))

<All keys matched successfully>

In [99]:
model.eval()
# 初始化损失
epoch_loss = 0
epoch_acc = 0
val_number = 0
predictions_test = []
labels_test = []
# 不计算梯度
with torch.no_grad():
    for batch in val_data_loader:
        text_id = batch['text_id'].to(device)
        label = batch['label'].to(device)
        predictions = model(text_id)
        loss = criterion(predictions, label)
        epoch_loss += loss.item()
        preds = predictions.max(1)[1]
        epoch_acc += ((preds == label).sum().item())
        val_number += label.size(0)
        predictions_test.extend(preds.tolist())
        labels_test.extend(label.tolist())
history[f'{model_name}_test_loss'].append(epoch_loss / len(val_data_loader))
history[f'{model_name}_test_accuracy'].append(epoch_acc / val_number)
if best_acc < (epoch_acc / val_number):
    best_acc = epoch_acc / val_number
print('测试Loss：', epoch_loss / len(val_data_loader), '，测试准确率：', epoch_acc / val_number)
print('-' * 20)

测试Loss： 1.120144334050917 ，测试准确率： 0.744709666497376
--------------------


In [35]:
import csv


filename = '33trainLoss.csv'

# 使用 'w' 模式打开文件
with open(filename, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    # 写入每一个预测值
    for prediction in history['BiLSTM_train_loss'][50:]:
        writer.writerow([prediction])

print('CSV文件写入完成')

CSV文件写入完成


In [36]:
import csv


filename = '33trainacc.csv'

# 使用 'w' 模式打开文件
with open(filename, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    # 写入每一个预测值
    for prediction in history['BiLSTM_train_accuracy'][50:]:
        writer.writerow([prediction])

print('CSV文件写入完成')

CSV文件写入完成


In [ ]:
model.eval()
# 初始化损失
epoch_loss = 0
epoch_acc = 0
val_number = 0
predictions_test = []
labels_test = []
# 不计算梯度
with torch.no_grad():
    for batch in val_data_loader:
        text_id = batch['text_id'].to(device)
        label = batch['label'].to(device)
        predictions = model(text_id)
        loss = criterion(predictions, label)
        epoch_loss += loss.item()
        preds = predictions.max(1)[1]
        epoch_acc += ((preds == label).sum().item())
        val_number += label.size(0)
        predictions_test.extend(preds.tolist())
        labels_test.extend(label.tolist())
history[f'{model_name}_test_loss'].append(epoch_loss / len(val_data_loader))
history[f'{model_name}_test_accuracy'].append(epoch_acc / val_number)
if best_acc < (epoch_acc / val_number):
    best_acc = epoch_acc / val_number
print('测试Loss：', epoch_loss / len(val_data_loader), '，测试准确率：', epoch_acc / val_number)
print('-' * 20)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

plt.style.use('ggplot')
plt.figure(figsize=(16,5),dpi=150)
plt.subplot(1,2,1)
plt.plot(history['BiLSTM_train_loss'],label='BiLSTM')
# plt.plot(history['CNNText_inception_test_loss'],label='CNNText_inception')
plt.title('Train Loss')

# 设置坐标轴刻度
plt.xticks(range(1,31))
plt.gca().xaxis.set_major_locator(ticker.MultipleLocator(4)) # 横坐标间隔为4

# 添加图例并设置位置
# plt.legend(frameon=True, loc='upper right', facecolor='white', bbox_to_anchor=(1.0, 1.0))


plt.subplot(1,2,2)
plt.plot(history['BiLSTM_train_accuracy'],label='BiLSTM')
# plt.plot(history['CNNText_inception_test_accuracy'],label='CNNText_inception')
plt.title('Train Accuracy')
plt.legend(frameon=True,loc='lower right',facecolor='white')
plt.xticks(range(1,31))
plt.gca().xaxis.set_major_locator(ticker.MultipleLocator(4)) # 横坐标间隔为5